<h3> MD ATIQUR RAHMAN </h3> <br>

<h1> Problem Statement </h1> 
<h4> <font color='maroon'>Predicting Average Movie Rating</font> </h4>


A movie's success or greatness can be measured based on its box office earnings and reviews/ratings that it gets from its critics and audiences after the release. Generally, it takes months to get a good amount of reviews and box office earnings to know whether the movie is a success or not. 

How can we tell whether a movie will be successful or not before being released in the theater? For example, most of the Cristopher Nolan movies (e.g., Interstellar, The Dark Knight Trilogy, Inception, etc.) were box office hits. They received a tremendous amount of positive reviews and high ratings from both critics and general audiences. Thus, my instincts tell me that a movie, either directed, written, or produced by him, will be a successful movie.

Is there a better way for us to capture these types of patterns mathematically (using ML algorithms) and predict whether a movie will be successful or not (what average rating a movie might get) before its release without relying on critics or our instincts? (not predicting success based on box office earnings. That can be a separate use case/model).

Therefore, we would like to explore a sample of movie meta data, find hidden patterns/insights and train an ML model:

- `understand what are the crucial factors that make a movie more successful than others, in other words, get a higher IMDB rating.`  

- `predict what a movie's average rating will be based on its available meta data before its release. `


An accurate model will help movie producers, makers, and business entities assess a movie's success before its release in theatre and make informed decisions. 

Data Descriptions: https://www.imdb.com/interfaces/

Data: https://datasets.imdbws.com/

<h3> <font color ='maroon'>  Goal: Predicting Average Movie Ratings </font><br> 


- <a href = '#1'> <font color='maroon'> Objectives </font> </a> <br> <br>

    - <a> understand what are the crucial factors that make a movie more successful than others, in other words, get a higher IMDB rating </a> <br> <br>

    - <a> predict what a movie's average rating will be based on its available meta data before its release. </a>


- <a href = '#2'> <font color='maroon'> Steps </font> </a> <br> <br>

    - <a> 1) Data </a> <br> <br>
        - <a> a) Data Preprocessing for Exploration and ML training<br> <br> </a> 

    - <a> 2) Data Exploration: Summarize the key findings and insights from these datasets <br> <br> </a>
    - <a> 3) Statistics and Feature Selection for Modeling  <br> <br> </a> 
    - <a> 4) Build Machine Learning Models to Predict a Movie's Average Rating  <br> <br> </a>
         - <a> Regression Models </a> <br> <br>
         - <a> Classification Models (Stretch goal) </a><br> <br>
         
    - <a> 5) Breakdown of Predictions  </a> <br> <br>

- <a href='#1'>1. Modules</a>
- <a href='#2'>2. Functions</a>
- <a href='#3'>3. Data</a>
    - <a href='#3.1'>3.1 Data Descriptions </a>
    - <a href='#3.2'>3.2 Extract WinRar Files </a>
    - <a href='#3.3'>3.3 Build Title and Rating DataFrame -> `title_basics_rating.csv` </a>
    - <a href='#3.4'>3.4 Build Crew, Titles, Ratings DataFrame -> `crew_titles_ratings.csv` </a>
    
- <a href='#4'>4. Feature Engineering</a>    
    - <a href='#4.1'>4.1 Build Crew Avg Summary -> `crew_avg_summary_df.csv` </a>
    - <a href='#4.2'>4.2 Merge Crew Summary with Crew_titles_ratings -> `crew_titles_ratings_summary.csv` </a>
    - <a href='#4.3'>4.3 Build Title Summary -> `title_summary.csv`</a>
    - <a href='#4.4'>4.4 Build Final DataFrame -> `movie_df_final.csv`</a>
  

# <a id='1'>1. Modules</a>

In [103]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_extraction.text import CountVectorizer

# Process raw WinRar files
import glob
import os
from pyunpack import Archive #pip install pyunpack, pip install patool
import gc

import psutil # check memory

# <a id='2'>2. Functionss</a>

In [2]:
# 0. Extract winrar files - run once 
def extract_winrar_files(DATA_DIR):
    '''
    Extracts winrar files. After extracting, removes the winrar files and keep the extracted files only.
    
    Input:
         DATA_DIR (str): path to winrar files
    '''
    # windows device - unzip raw data
    os.chdir(DATA_DIR)
    files = glob.glob(DATA_DIR+'*')
    for file in files:
        file_split = file.split("\\")[1]
        print(file_split)
        file_split_path = os.getcwd()+"\\"
        Archive(file_split).extractall(file_split_path)
        try:
            os.remove(file_split)
        except OSError as e:
            print("Error: %s : %s" % (file_split, e.strerror))

# <a id='3'>3. Data</a>

## <a id='3.1'>3.1  Data Descriptions</a>

IMDb Dataset Details

Each dataset is contained in a gzipped, tab-separated-values (TSV) formatted file in the UTF-8 character set. The first line in each file contains headers that describe what is in each column. A ‘\N’ is used to denote that a particular field is missing or null for that title/name. The available datasets are as follows:

#### A. title.ratings.tsv.gz – Contains the IMDb rating and votes information for titles
`tconst (string)` - alphanumeric unique identifier of the title<br>
`averageRating` – weighted average of all the individual user ratings<br>
`numVotes` - number of votes the title has received<br><br>


#### B. title.basics.tsv.gz - Contains the following information for titles:
`tconst (string)` - alphanumeric unique identifier of the title <br>
`titleType (string)` – the type/format of the title (e.g. movie, short, tvseries, tvepisode, video, etc)<br>
`primaryTitle (string)` – the more popular title / the title used by the filmmakers on promotional materials at the point of release<br>
`originalTitle (string)` - original title, in the original language<br>
`isAdult (boolean)` - 0: non-adult title; 1: adult title<br>
`startYear (YYYY)` – represents the release year of a title. In the case of TV Series, it is the series start year<br>
`endYear (YYYY)` – TV Series end year. ‘\N’ for all other title types<br>
`runtimeMinutes` – primary runtime of the title, in minutes<br>
`genres (string array)` – includes up to three genres associated with the title<br><br>


#### C. title.principals.tsv.gz – Contains the principal cast/crew for titles
`tconst (string)` - alphanumeric unique identifier of the title<br>
`ordering (integer)` – a number to uniquely identify rows for a given titleId<br>
`nconst (string)` - alphanumeric unique identifier of the name/person<br>
`category (string)` - the category of job that person was in<br>
`job (string)` - the specific job title if applicable, else '\N'<br>
`characters (string)` - the name of the character played if applicable, else '\N'<br><br>

#### D. name.basics.tsv.gz – Contains the following information for names:
`nconst (string)` - alphanumeric unique identifier of the name/person<br>
`primaryName (string)`– name by which the person is most often credited<br>
`birthYear` – in YYYY format<br>
`deathYear` – in YYYY format if applicable, else '\N'<br>
`primaryProfession (array of strings)`– the top-3 professions of the person<br>
`knownForTitles (array of tconsts)` – titles the person is known for<br><br>



#### E. title.akas.tsv.gz - Contains the following information for titles:

`titleId (string)` - a tconst, an alphanumeric unique identifier of the title <br>
`ordering (integer)` – a number to uniquely identify rows for a given titleId <br>
`title (string)` – the localized title <br>
`region (string)` - the region for this version of the title <br>
`language (string)` - the language of the title <br>
`types (array)` - Enumerated set of attributes for this alternative title. One or more of the following: "alternative", "dvd", "festival", "tv", "video", "working", "original", "imdbDisplay". New values may be added in the future without warning <br>
`attributes (array)` - Additional terms to describe this alternative title, not enumerated <br>
`isOriginalTitle (boolean)` – 0: not original title; 1: original title <br> <br>

#### F. title.crew.tsv.gz – Contains the director and writer information for all the titles in IMDb. Fields include:
`tconst (string)` - alphanumeric unique identifier of the title<br>
`directors (array of nconsts)` - director(s) of the given title<br>
`writers (array of nconsts)` – writer(s) of the given title<br><br>

####  G. title.episode.tsv.gz – Contains the tv episode information. Fields include:
`tconst (string)` - alphanumeric identifier of episode<br>
`parentTconst (string)` - alphanumeric identifier of the parent TV Series<br>
`seasonNumber (integer)` – season number the episode belongs to<br>
`episodeNumber (integer)` – episode number of the tconst in the TV series<br><br>

In [ ]:
DATA_DIR = "../../data/raw/"

## <a id='3.2'>3.2  Extract WinRar Files</a>

In [ ]:
# 0. Extract winrar files - run once 
# extract_winrar_files(DATA_DIR)

In [ ]:
# collecting garbage
collected = gc.collect()
print("Garbage collector: collected",
          "%d objects." % collected)

## <a id='3.3'>3.3 Build Title and Rating DataFrame -> `title_basics_rating.csv`</a>

### A) title.ratings.tsv

In [59]:
DATA_DIR = "../data/raw/"

In [60]:
psutil.virtual_memory()

svmem(total=17011007488, available=5709369344, percent=66.4, used=11301638144, free=5709369344)

In [61]:
title_ratings = pd.read_table(DATA_DIR+"title.ratings.tsv")

print(title_ratings.shape)
print(title_ratings.duplicated().any())
print(len(title_ratings['tconst'].unique()))
print(title_ratings.isna().sum().sum())

(1135764, 3)
False
1135764
0


In [62]:
title_ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.6,1695
1,tt0000002,6.0,210
2,tt0000003,6.5,1442
3,tt0000004,6.1,122
4,tt0000005,6.1,2242


### B) title.basics.tsv

In [63]:
title_basics = pd.read_table(DATA_DIR+"title.basics.tsv")

print(title_basics.shape)
print(title_basics.duplicated().any())
print(len(title_basics['tconst'].unique()))
print(title_basics.isna().sum().sum())

C:\Users\MDAtiqurRahman\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3249: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


(7770807, 9)
False
7770807
32


In [64]:
title_basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [65]:
title_basics['titleType'].value_counts()

tvEpisode       5652753
short            802859
movie            573095
video            300773
tvSeries         204506
tvMovie          130681
tvMiniSeries      36646
tvSpecial         31965
videoGame         27675
tvShort            9851
radioSeries           1
episode               1
audiobook             1
Name: titleType, dtype: int64

### C) Keep movie data only, drop everything else

In [66]:
title_basics = title_basics[title_basics['titleType'] == 'movie']

In [67]:
title_basics.shape

(573095, 9)

### D)  Drop all records where `runtimeMinutes` or `genres` == "\N" 

In [68]:
title_basics = title_basics[~title_basics['runtimeMinutes'].str.contains(r'\\N', regex=True)]

In [69]:
title_basics = title_basics[~title_basics['genres'].str.contains(r'\\N', regex=True)]

In [70]:
title_basics.shape

(337625, 9)

In [71]:
title_basics.isna().sum()

tconst            0
titleType         0
primaryTitle      0
originalTitle     0
isAdult           0
startYear         0
endYear           0
runtimeMinutes    0
genres            0
dtype: int64

### E) Drop unnecessary columns

In [72]:
title_basics = title_basics.drop(['titleType', 'originalTitle', 'startYear', 'endYear'], axis=1)

In [73]:
title_basics.head()

,tconst,primaryTitle,isAdult,runtimeMinutes,genres
570,tt0000574,The Story of the Kelly Gang,0,70,"Action,Adventure,Biography"
672,tt0000679,The Fairylogue and Radio-Plays,0,120,"Adventure,Fantasy"
1172,tt0001184,Don Juan de Serrallonga,0,58,"Adventure,Drama"
1246,tt0001258,The White Slave Trade,0,45,Drama
1273,tt0001285,The Life of Moses,0,50,"Biography,Drama,Family"


In [74]:
title_basics.shape

(337625, 5)

### F) Merge A and C - ratings with title_basics

This will keep the only movies that have rating information

In [75]:
title_basics_rating = title_basics.merge(title_ratings, on='tconst', how='left')

In [76]:
title_basics_rating.shape

(337625, 7)

In [78]:
title_basics_rating = title_basics_rating.dropna()

In [79]:
title_basics_rating.shape

(228790, 7)

In [81]:
title_basics_rating.head()

,tconst,primaryTitle,isAdult,runtimeMinutes,genres,averageRating,numVotes
0,tt0000574,The Story of the Kelly Gang,0,70,"Action,Adventure,Biography",6.1,625.0
1,tt0000679,The Fairylogue and Radio-Plays,0,120,"Adventure,Fantasy",5.2,35.0
2,tt0001184,Don Juan de Serrallonga,0,58,"Adventure,Drama",3.4,11.0
3,tt0001258,The White Slave Trade,0,45,Drama,5.7,101.0
4,tt0001285,The Life of Moses,0,50,"Biography,Drama,Family",5.4,29.0


### G) Keep Title Ids of interest

In [82]:
title_basics_rating_ids = title_basics_rating['tconst'].unique()

### H) Save in interim folder and delete cache

In [83]:
# collecting garbage
collected = gc.collect()
print("Garbage collector: collected",
          "%d objects." % collected)
      
title_basics_rating.to_csv("../data/interim/title_basics_rating.csv", index=False)

Garbage collector: collected 169 objects.


In [85]:
title_basics_rating.head()

,tconst,primaryTitle,isAdult,runtimeMinutes,genres,averageRating,numVotes
0,tt0000574,The Story of the Kelly Gang,0,70,"Action,Adventure,Biography",6.1,625.0
1,tt0000679,The Fairylogue and Radio-Plays,0,120,"Adventure,Fantasy",5.2,35.0
2,tt0001184,Don Juan de Serrallonga,0,58,"Adventure,Drama",3.4,11.0
3,tt0001258,The White Slave Trade,0,45,Drama,5.7,101.0
4,tt0001285,The Life of Moses,0,50,"Biography,Drama,Family",5.4,29.0


In [86]:
del title_ratings
del title_basics
del title_basics_rating

## <a id='3.4'>3.4 Build Crew, Titles, Ratings DataFrame -> `crew_titles_ratings.csv`</a>

### A) title.principals.tsv  – Contains the principal cast/crew for titles

In [87]:
title_principals = pd.read_table(DATA_DIR+"title.principals.tsv")

print(title_principals.shape)
print(title_principals.duplicated().any())
print(len(title_principals['tconst'].unique()))
print(title_principals.isna().sum().sum())

(43925408, 6)
False
7059190
0


In [88]:
title_principals.shape

(43925408, 6)

In [89]:
title_principals.head()

,tconst,ordering,nconst,category,job,characters
0,tt0000001,1,nm1588970,self,\N,"[""Self""]"
1,tt0000001,2,nm0005690,director,\N,\N
2,tt0000001,3,nm0374658,cinematographer,director of photography,\N
3,tt0000002,1,nm0721526,director,\N,\N
4,tt0000002,2,nm1335271,composer,\N,\N


### B) Filtrer out unneccessary title_basics_rating_ids

In [90]:
title_principals = title_principals[title_principals['tconst'].isin(title_basics_rating_ids)]

In [92]:
title_principals.shape

(2069262, 6)

### C) name.basics.tsv

In [91]:
name_basics = pd.read_table(DATA_DIR+"name.basics.tsv")

print(name_basics.shape)
print(name_basics.duplicated().any())
print(len(name_basics['nconst'].unique()))
print(name_basics.isna().sum().sum())

(10832003, 6)
False
10832003
2288780


### D) Merge A and C - ratings with title_basics

In [96]:
name_basics[name_basics['nconst'] == 'nm0675239']

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
637826,nm0675239,Orrie Perry,1888,1950,"cinematographer,actor","tt0000335,tt0322792,tt0336770,tt0000574"


In [98]:
title_principals[title_principals['nconst'] == 'nm0675239']

,tconst,ordering,nconst,category,job,characters
1040,tt0000574,10,nm0675239,cinematographer,director of photography,\N
1587750,tt0203943,6,nm0675239,cinematographer,\N,\N


In [99]:
name_basics_title_principals = pd.merge(name_basics, title_principals, on='nconst', how='left')

In [100]:
name_basics_title_principals.shape

(12149281, 11)

In [101]:
name_basics_title_principals = name_basics_title_principals[name_basics_title_principals['tconst'].isin(title_basics_rating_ids)]

In [102]:
name_basics_title_principals.shape

(2068791, 11)

In [104]:
name_basics_title_principals.tail()

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles,tconst,ordering,category,job,characters
12149059,nm9993485,Augusto Cortesi,\N,\N,actor,"tt9534676,tt9615718",tt9615718,4.0,actor,\N,"[""Robot Leader""]"
12149068,nm9993494,Amjad Ali,\N,\N,writer,tt7722258,tt7722258,7.0,writer,screenplay and dialogue,\N
12149181,nm9993616,Ryan Mac Lennan,\N,\N,actor,tt4844148,tt4844148,4.0,actor,\N,"[""Mick Owen""]"
12149213,nm9993650,Marcin Balcerak,\N,\N,actor,tt8739208,tt8739208,4.0,actor,\N,"[""Guardian""]"
12149256,nm9993693,Anketa Maharana,\N,\N,actress,"tt9815360,tt14087492,tt8737752",tt9815360,2.0,actress,\N,"[""Anupama""]"


### E) Data Not Useful

#### title.akas.tsv - not useful

In [104]:
# title_akas = pd.read_table(DATA_DIR+"title.akas.tsv")

#### title.crew.tsv – Contains the director and writer information for all the titles in IMDb. Not useful

In [106]:
title_crew = pd.read_table(DATA_DIR+"title.crew.tsv")
title_crew.shape
title_crew.head()
title_crew.duplicated().any()
len(title_crew['tconst'].unique())
title_crew.isna().sum().sum()
del title_crew

#### title.episode.tsv – Contains the tv episode information. - not useful

In [ ]:
title_episode = pd.read_table(DATA_DIR+"title.episode.tsv")
title_episode.shape

In [ ]:
title_episode.head()

In [ ]:
title_episode.duplicated().any()

In [ ]:
len(title_episode['tconst'].unique())

In [ ]:
title_episode.isna().sum().sum()

#### How many regions did the movie get released into? Is there any correlation between num_regions and avg. rating ?

In [ ]:
#title_akas[title_akas['titleId'] == 'tt0000005']
#title_akas.shape
#title_akas_max_region = title_akas.sort_values('ordering', ascending=False).groupby('titleId').head(1)
#len(title_akas_max_region['titleId'].unique())
#title_akas_max_region = title_akas_max_region[['titleId','ordering']]
#title_akas_max_region.columns = ['titleId', 'num_regions']
#title_akas_max_region
#df_merged = pd.merge(df_merged, title_akas_max_region, left_on='tconst', right_on='titleId', how='left')
#df_merged.isna().sum()
#df_merged[['num_regions', 'averageRating']].corr()

# del title_akas

# # collecting garbage
# collected = gc.collect()
# print("Garbage collector: collected",
#           "%d objects." % collected)
#psutil.virtual_memory()

No correlation between num_regions and averageRating

### F) Merge `name_basics_title_principals` and `title_basics_rating`

In [113]:
title_basics_rating.head(1)

,tconst,primaryTitle,isAdult,runtimeMinutes,genres,averageRating,numVotes
0,tt0000574,The Story of the Kelly Gang,0,70,"Action,Adventure,Biography",6.1,625.0


In [114]:
name_basics_title_principals.head(1)

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles,tconst,ordering,category,job,characters
0,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0031983,tt0072308,tt0053137,tt0050419",tt0025164,1.0,actor,\N,"[""Guy Holden""]"


In [120]:
crew_titles_ratings = pd.merge(title_basics_rating, name_basics_title_principals, on='tconst', how='left')

In [121]:
crew_titles_ratings.shape

(2068974, 17)

In [122]:
crew_titles_ratings.tail()

,tconst,primaryTitle,isAdult,runtimeMinutes,genres,averageRating,numVotes,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles,ordering,category,job,characters
2068969,tt9916538,Kuambil Lagi Hatiku,0,123,Drama,8.3,6.0,nm4457074,Azhar Kinoi Lubis,\N,\N,"director,assistant_director","tt9916538,tt8517144,tt1930290,tt5774688",5.0,director,\N,\N
2068970,tt9916538,Kuambil Lagi Hatiku,0,123,Drama,8.3,6.0,nm4700236,Dave Lumenta,\N,\N,"composer,actor,music_department","tt3777170,tt9916538,tt7541708,tt2073058",10.0,composer,music director,\N
2068971,tt9916538,Kuambil Lagi Hatiku,0,123,Drama,8.3,6.0,nm4843252,Arief Ash Siddiq,\N,\N,"writer,producer,actor","tt9916538,tt12741494,tt12484670,tt2172015",6.0,writer,script writer,\N
2068972,tt9916538,Kuambil Lagi Hatiku,0,123,Drama,8.3,6.0,nm4900525,Rino Sarjono,\N,\N,"writer,producer,actor","tt11554776,tt9916538,tt2190357,tt8712194",7.0,writer,script writer,\N
2068973,tt9916538,Kuambil Lagi Hatiku,0,123,Drama,8.3,6.0,nm8678236,Lala Karmela,\N,\N,actress,"tt6380556,tt8060288,tt9916538,tt6503596",1.0,actress,\N,"[""Sinta""]"


In [118]:
psutil.virtual_memory()

svmem(total=17011007488, available=3228131328, percent=81.0, used=13782876160, free=3228131328)

In [123]:
crew_titles_ratings.columns

Index(['tconst', 'primaryTitle', 'isAdult', 'runtimeMinutes', 'genres',
       'averageRating', 'numVotes', 'nconst', 'primaryName', 'birthYear',
       'deathYear', 'primaryProfession', 'knownForTitles', 'ordering',
       'category', 'job', 'characters'],
      dtype='object')

In [125]:
crew_titles_ratings = crew_titles_ratings[['tconst', 'primaryTitle', 'isAdult', 'runtimeMinutes', 'genres',
       'averageRating', 'numVotes', 'nconst', 'primaryName', 'ordering','category', ]]

In [126]:
crew_titles_ratings.head()

,tconst,primaryTitle,isAdult,runtimeMinutes,genres,averageRating,numVotes,nconst,primaryName,ordering,category
0,tt0000574,The Story of the Kelly Gang,0,70,"Action,Adventure,Biography",6.1,625.0,nm0170118,Bella Cola,4.0,actress
1,tt0000574,The Story of the Kelly Gang,0,70,"Action,Adventure,Biography",6.1,625.0,nm0317210,W.A. Gibson,6.0,producer
2,tt0000574,The Story of the Kelly Gang,0,70,"Action,Adventure,Biography",6.1,625.0,nm0425854,Millard Johnson,7.0,producer
3,tt0000574,The Story of the Kelly Gang,0,70,"Action,Adventure,Biography",6.1,625.0,nm0675239,Orrie Perry,10.0,cinematographer
4,tt0000574,The Story of the Kelly Gang,0,70,"Action,Adventure,Biography",6.1,625.0,nm0846879,Charles Tait,5.0,director


In [127]:
crew_titles_ratings['category'].unique()

array(['actress', 'producer', 'cinematographer', 'director', 'actor',
       'composer', 'writer', 'production_designer', 'editor', 'self',
       'archive_footage', nan, 'archive_sound'], dtype=object)

In [128]:
crew_titles_ratings['category'].value_counts()

actor                  535215
actress                322945
writer                 245900
director               238958
producer               202891
composer               164386
cinematographer        159527
editor                 101237
self                    67594
production_designer     22781
archive_footage          7324
archive_sound              33
Name: category, dtype: int64

In [129]:
crew_titles_ratings.isna().sum()

tconst              0
primaryTitle        0
isAdult             0
runtimeMinutes      0
genres              0
averageRating       0
numVotes            0
nconst            183
primaryName       183
ordering          183
category          183
dtype: int64

In [131]:
crew_titles_ratings.shape

(2068974, 11)

In [132]:
crew_titles_ratings = crew_titles_ratings.dropna()

In [136]:
crew_titles_ratings.shape

(2068791, 11)

In [137]:
crew_titles_ratings.head()

,tconst,primaryTitle,isAdult,runtimeMinutes,genres,averageRating,numVotes,nconst,primaryName,ordering,category
0,tt0000574,The Story of the Kelly Gang,0,70,"Action,Adventure,Biography",6.1,625.0,nm0170118,Bella Cola,4.0,actress
1,tt0000574,The Story of the Kelly Gang,0,70,"Action,Adventure,Biography",6.1,625.0,nm0317210,W.A. Gibson,6.0,producer
2,tt0000574,The Story of the Kelly Gang,0,70,"Action,Adventure,Biography",6.1,625.0,nm0425854,Millard Johnson,7.0,producer
3,tt0000574,The Story of the Kelly Gang,0,70,"Action,Adventure,Biography",6.1,625.0,nm0675239,Orrie Perry,10.0,cinematographer
4,tt0000574,The Story of the Kelly Gang,0,70,"Action,Adventure,Biography",6.1,625.0,nm0846879,Charles Tait,5.0,director


### G) Save in interim folder and delete cache

In [138]:
crew_titles_ratings.to_csv("../data/interim/crew_titles_ratings.csv", index=False)

In [ ]:
del crew_titles_ratings

In [ ]:
del name_basics_title_principals

# <a id='4'>4. Feature Engineering </a>

### 4.1) Build Crew Avg Summary -> `crew_avg_summary_df.csv`

In [3]:
crew_titles_ratings = pd.read_csv("../data/interim/crew_titles_ratings.csv")

In [4]:
title_basics_ratings = pd.read_csv("../data/interim/title_basics_rating.csv")

In [17]:
def crew_avg_summary(title_id):
    '''
    Returns a summary of the crew for a given title. The summary excludes the given title from the summary.
    
    '''
    crew_list              = crew_titles_ratings[crew_titles_ratings['tconst'] == title_id]['nconst'].unique()
    crew_other_titles      = crew_titles_ratings[(crew_titles_ratings['nconst'].isin(crew_list))]
    crew_other_titles      = crew_other_titles[crew_other_titles['tconst'] != title_id]
    num_asscociated_titles = len(crew_other_titles['tconst'].unique())
    avg = crew_other_titles[['averageRating', 'numVotes']].mean()
    
    return title_id, num_asscociated_titles, avg['averageRating'], avg['numVotes']
    

In [18]:
title_id = "tt0111161"
crew_avg_summary(title_id)

('tt0111161', 321, 6.511445783132533, 103147.1686746988)

In [37]:
title_ids = crew_titles_ratings['tconst'].unique()
len(title_ids)

228607

In [44]:
crew_avg_summary_list = []
count = 1
LOG_EVERY_N = 100
for title_id in title_ids:
    
    if (count % LOG_EVERY_N) == 0:
        print(count)
    title_id, num_asscociated_titles, averageRating, numVotes = crew_avg_summary(title_id)
    
    crew_avg_summary_list.append((title_id, num_asscociated_titles, averageRating, numVotes))
    count +=1

crew_avg_summary_df = pd.DataFrame(crew_avg_summary_list, columns = ['tconst','crew_titles_asscociated', 'crew_titles_avgRating', 'crew_titles_avgNumVotes'])
crew_avg_summary_df.to_csv("../data/interim/crew_avg_summary_df.csv", index=False)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
1850

133000
133100
133200
133300
133400
133500
133600
133700
133800
133900
134000
134100
134200
134300
134400
134500
134600
134700
134800
134900
135000
135100
135200
135300
135400
135500
135600
135700
135800
135900
136000
136100
136200
136300
136400
136500
136600
136700
136800
136900
137000
137100
137200
137300
137400
137500
137600
137700
137800
137900
138000
138100
138200
138300
138400
138500
138600
138700
138800
138900
139000
139100
139200
139300
139400
139500
139600
139700
139800
139900
140000
140100
140200
140300
140400
140500
140600
140700
140800
140900
141000
141100
141200
141300
141400
141500
141600
141700
141800
141900
142000
142100
142200
142300
142400
142500
142600
142700
142800
142900
143000
143100
143200
143300
143400
143500
143600
143700
143800
143900
144000
144100
144200
144300
144400
144500
144600
144700
144800
144900
145000
145100
145200
145300
145400
145500
145600
145700
145800
145900
146000
146100
146200
146300
146400
146500
146600
146700
146800
146900
147000
147100
147200

In [46]:
crew_avg_summary_df.head()

,tconst,crew_titles_asscociated,crew_titles_avgRating,crew_titles_avgNumVotes
0,tt0000574,1,6.200000,11.000000
1,tt0000679,22,5.181818,11092.772727
2,tt0001184,5,6.080000,17.000000
3,tt0001258,18,5.756522,70.086957
4,tt0001285,27,6.234483,40.448276


In [47]:
crew_avg_summary_df.isna().sum()

tconst                        0
crew_titles_asscociated       0
crew_titles_avgRating      5318
crew_titles_avgNumVotes    5318
dtype: int64

In [93]:
# crew = crew_titles_ratings.groupby(['nconst'], as_index=False).agg({"tconst"        : "count",
#                                                                     "averageRating" : "mean",
#                                                                     "numVotes"      : "mean",
#                                                                     "ordering"      : "mean"}).rename(columns={"tconst"       : "crew_titles_asscociated",
#                                                                                                                "averageRating": "crew_titles_avgRating",
#                                                                                                                "numVotes"     : "crew_titles_avgNumVotes",
#                                                                                                                "ordering"     : "crew_titles_avgOrdering"})

### 4.2 Merge Crew Summary with Crew_titles_ratings -> `crew_titles_ratings_summary.csv`

In [48]:
crew_titles_ratings.shape

(2068791, 11)

In [49]:
crew_avg_summary_df.shape

(228607, 4)

In [53]:
crew_titles_ratings_summary = pd.merge(crew_titles_ratings, crew_avg_summary_df, on='tconst', how='left')

In [54]:
crew_titles_ratings_summary.head(2)

,tconst,primaryTitle,isAdult,runtimeMinutes,genres,averageRating,numVotes,nconst,primaryName,ordering,category,crew_titles_asscociated,crew_titles_avgRating,crew_titles_avgNumVotes
0,tt0000574,The Story of the Kelly Gang,0,70,"Action,Adventure,Biography",6.1,625.0,nm0170118,Bella Cola,4.0,actress,1,6.2,11.0
1,tt0000574,The Story of the Kelly Gang,0,70,"Action,Adventure,Biography",6.1,625.0,nm0317210,W.A. Gibson,6.0,producer,1,6.2,11.0


In [55]:
crew_titles_ratings_summary.shape

(2068791, 14)

In [57]:
crew_titles_ratings_summary.isna().sum()

tconst                         0
primaryTitle                   0
isAdult                        0
runtimeMinutes                 0
genres                         0
averageRating                  0
numVotes                       0
nconst                         0
primaryName                    0
ordering                       0
category                       0
crew_titles_asscociated        0
crew_titles_avgRating      30939
crew_titles_avgNumVotes    30939
dtype: int64

In [59]:
crew_titles_ratings_summary[crew_titles_ratings_summary['tconst'] == 'tt0025164'].head()

,tconst,primaryTitle,isAdult,runtimeMinutes,genres,averageRating,numVotes,nconst,primaryName,ordering,category,crew_titles_asscociated,crew_titles_avgRating,crew_titles_avgNumVotes
52855,tt0025164,The Gay Divorcee,0,107,"Comedy,Musical,Romance",7.5,7098.0,nm0000001,Fred Astaire,1.0,actor,382,6.470073,1618.856448
52856,tt0025164,The Gay Divorcee,0,107,"Comedy,Musical,Romance",7.5,7098.0,nm0001677,Ginger Rogers,2.0,actress,382,6.470073,1618.856448
52857,tt0025164,The Gay Divorcee,0,107,"Comedy,Musical,Romance",7.5,7098.0,nm0002143,Edward Everett Horton,4.0,actor,382,6.470073,1618.856448
52858,tt0025164,The Gay Divorcee,0,107,"Comedy,Musical,Romance",7.5,7098.0,nm0103567,Alice Brady,3.0,actress,382,6.470073,1618.856448
52859,tt0025164,The Gay Divorcee,0,107,"Comedy,Musical,Romance",7.5,7098.0,nm0388755,Samuel Hoffenstein,8.0,writer,382,6.470073,1618.856448


In [60]:
crew_titles_ratings_summary = crew_titles_ratings_summary[crew_titles_ratings_summary['category'] != 'self']

In [111]:
crew_titles_ratings_summary['category'].value_counts()

actor                  535215
actress                322945
writer                 245900
director               238958
producer               202891
composer               164386
cinematographer        159527
editor                 101237
production_designer     22781
archive_footage          7324
archive_sound              33
Name: category, dtype: int64

In [62]:
crew_titles_ratings_summary.isna().sum()

tconst                         0
primaryTitle                   0
isAdult                        0
runtimeMinutes                 0
genres                         0
averageRating                  0
numVotes                       0
nconst                         0
primaryName                    0
ordering                       0
category                       0
crew_titles_asscociated        0
crew_titles_avgRating      27656
crew_titles_avgNumVotes    27656
dtype: int64

In [63]:
crew_titles_ratings_summary = crew_titles_ratings_summary.dropna()

In [66]:
crew_titles_ratings_summary.to_csv("../data/interim/crew_titles_ratings_summary.csv", index=False)

In [79]:
crew_titles_ratings_summary.head()

,tconst,primaryTitle,isAdult,runtimeMinutes,genres,averageRating,numVotes,nconst,primaryName,ordering,category,crew_titles_asscociated,crew_titles_avgRating,crew_titles_avgNumVotes
0,tt0000574,The Story of the Kelly Gang,0,70,"Action,Adventure,Biography",6.1,625.0,nm0170118,Bella Cola,4.0,actress,1,6.2,11.0
1,tt0000574,The Story of the Kelly Gang,0,70,"Action,Adventure,Biography",6.1,625.0,nm0317210,W.A. Gibson,6.0,producer,1,6.2,11.0
2,tt0000574,The Story of the Kelly Gang,0,70,"Action,Adventure,Biography",6.1,625.0,nm0425854,Millard Johnson,7.0,producer,1,6.2,11.0
3,tt0000574,The Story of the Kelly Gang,0,70,"Action,Adventure,Biography",6.1,625.0,nm0675239,Orrie Perry,10.0,cinematographer,1,6.2,11.0
4,tt0000574,The Story of the Kelly Gang,0,70,"Action,Adventure,Biography",6.1,625.0,nm0846879,Charles Tait,5.0,director,1,6.2,11.0


In [137]:
crew_titles_ratings_summary.shape

(2001197, 15)

In [138]:
len(crew_titles_ratings_summary['tconst'].unique())

228302

In [139]:
crew_titles_ratings_summary.head(1)

,tconst,primaryTitle,isAdult,runtimeMinutes,genres,averageRating,numVotes,nconst,primaryName,ordering,category,crew_titles_asscociated,crew_titles_avgRating,crew_titles_avgNumVotes,crew_titles_avgOrdering
0,tt0000574,The Story of the Kelly Gang,0,70,"Action,Adventure,Biography",6.1,625.0,nm0170118,Bella Cola,4.0,actress,1,6.1,625.0,4.0


### 4.3 Build Title Summary -> `title_summary.csv`

In [81]:
title_summary= crew_titles_ratings_summary.groupby(['tconst'], as_index=False).agg({
                                                                           "crew_titles_asscociated"   : "mean",
                                                                           "crew_titles_avgRating"     : "mean",
                                                                           "crew_titles_avgNumVotes"   : "mean"})

In [96]:
title_summary.head()

,tconst,crew_titles_asscociated,crew_titles_avgRating,crew_titles_avgNumVotes
0,tt0000574,1,6.200000,11.000000
1,tt0000679,22,5.181818,11092.772727
2,tt0001184,5,6.080000,17.000000
3,tt0001258,18,5.756522,70.086957
4,tt0001285,27,6.234483,40.448276


In [83]:
crew_titles_ratings_summary[crew_titles_ratings_summary['tconst'] == 'tt0000574']['crew_titles_avgNumVotes'].mean()

11.0

In [84]:
title_summary.to_csv("../data/interim/title_summary.csv", index=False)

### 4.4 Build Final DataFrame -> `movie_df_final.csv`

In [85]:
title_basics_ratings = pd.read_csv("../data/interim/title_basics_rating.csv")

In [153]:
title_summary        = pd.read_csv("../data/interim/title_summary.csv")

In [86]:
title_basics_ratings.head()

,tconst,primaryTitle,isAdult,runtimeMinutes,genres,averageRating,numVotes
0,tt0000574,The Story of the Kelly Gang,0,70,"Action,Adventure,Biography",6.1,625.0
1,tt0000679,The Fairylogue and Radio-Plays,0,120,"Adventure,Fantasy",5.2,35.0
2,tt0001184,Don Juan de Serrallonga,0,58,"Adventure,Drama",3.4,11.0
3,tt0001258,The White Slave Trade,0,45,Drama,5.7,101.0
4,tt0001285,The Life of Moses,0,50,"Biography,Drama,Family",5.4,29.0


In [97]:
title_basics_ratings.shape

(228790, 7)

In [88]:
title_summary.head()

,tconst,crew_titles_asscociated,crew_titles_avgRating,crew_titles_avgNumVotes
0,tt0000574,1,6.200000,11.000000
1,tt0000679,22,5.181818,11092.772727
2,tt0001184,5,6.080000,17.000000
3,tt0001258,18,5.756522,70.086957
4,tt0001285,27,6.234483,40.448276


In [98]:
title_summary.shape

(223042, 4)

In [90]:
df_merged = pd.merge(title_basics_ratings, title_summary, on='tconst', how="inner")

In [93]:
df_merged.isna().sum()

tconst                     0
primaryTitle               0
isAdult                    0
runtimeMinutes             0
genres                     0
averageRating              0
numVotes                   0
crew_titles_asscociated    0
crew_titles_avgRating      0
crew_titles_avgNumVotes    0
dtype: int64

In [95]:
df_merged[df_merged['primaryTitle'] == 'Inception']

,tconst,primaryTitle,isAdult,runtimeMinutes,genres,averageRating,numVotes,crew_titles_asscociated,crew_titles_avgRating,crew_titles_avgNumVotes
141505,tt1375666,Inception,0,148,"Action,Adventure,Sci-Fi",8.8,2092810.0,221,6.704292,193239.67382


In [102]:
df_merged.head()

,tconst,primaryTitle,isAdult,runtimeMinutes,genres,averageRating,numVotes,crew_titles_asscociated,crew_titles_avgRating,crew_titles_avgNumVotes
0,tt0000574,The Story of the Kelly Gang,0,70,"Action,Adventure,Biography",6.1,625.0,1,6.200000,11.000000
1,tt0000679,The Fairylogue and Radio-Plays,0,120,"Adventure,Fantasy",5.2,35.0,22,5.181818,11092.772727
2,tt0001184,Don Juan de Serrallonga,0,58,"Adventure,Drama",3.4,11.0,5,6.080000,17.000000
3,tt0001258,The White Slave Trade,0,45,Drama,5.7,101.0,18,5.756522,70.086957
4,tt0001285,The Life of Moses,0,50,"Biography,Drama,Family",5.4,29.0,27,6.234483,40.448276


In [100]:
df_merged.corr()

,isAdult,runtimeMinutes,averageRating,numVotes,crew_titles_asscociated,crew_titles_avgRating,crew_titles_avgNumVotes
isAdult,1.000000,-0.055916,-0.046331,-0.013980,0.034793,-0.085467,-0.039269
runtimeMinutes,-0.055916,1.000000,0.037671,0.064221,0.092691,0.030643,0.057457
averageRating,-0.046331,0.037671,1.000000,0.079949,-0.035502,0.521282,0.036903
numVotes,-0.013980,0.064221,0.079949,1.000000,0.062789,0.035027,0.497730
crew_titles_asscociated,0.034793,0.092691,-0.035502,0.062789,1.000000,-0.017774,0.053158
crew_titles_avgRating,-0.085467,0.030643,0.521282,0.035027,-0.017774,1.000000,0.057403
crew_titles_avgNumVotes,-0.039269,0.057457,0.036903,0.497730,0.053158,0.057403,1.000000


In [101]:
df_merged.to_csv("../data/processed/movie_df_final.csv", index=False)